https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#class_weights

https://medium.com/@zergtant/use-weighted-loss-function-to-solve-imbalanced-data-classification-problems-749237f38b75

https://github.com/kathrinse/TabSurvey/blob/main/utils/load_data.py

In [1]:
import sys
import tensorflow as tf
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

sys.path.append("../")

In [2]:
print(tf.__version__)

2.14.1


In [46]:
'''
filepath_train = 'adult_train.csv'
filepath_test = 'adult_test.csv'
df_train = pd.read_csv(filepath_train)
df_test = pd.read_csv(filepath_test)

columns = df_train.columns
target = 'income'

cat_columns = columns[[1,3,5,6,7,8,9,13,14]]
num_columns = columns[[0,2,4,10,11,12]]

print(df_train.shape)
print(df_test.shape)
'''

features = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital-status', 'occupation',
            'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']
target = 'income'
columns = features + [target]

filepath = 'adult.data'
df = pd.read_csv(filepath, names=columns)

columns = df.columns
target = 'income'

cat_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
num_columns = ['age', 'fnlwgt', 'education_num', 'capital-gain', 'capital-loss', 'hours-per-week']

print(df.shape)

(32561, 15)


Replace all ' ?' with np.NaN, and remove rows with missing values (np.NaN) or
replace all with 0

In [47]:
df.replace(' ?', np.NaN, inplace=True)
#df.dropna(inplace=True)
df.fillna(0, inplace=True)

Check for missing values

In [48]:
print(np.sum(df.isna(), axis=0))

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64


In [49]:
from encoder import ColumnEncoder

fe_params = {'columns_cat': cat_columns, 'columns_num': num_columns}

fe = ColumnEncoder(**fe_params)
fe.fit(df)
np_data = fe.transform(df)
print(np_data.shape)

TypeError: Encoders require their input argument must be uniformly strings or numbers. Got ['int', 'str']

In [30]:
fe.col_indices

{'age': [0, 1],
 'workclass': [1, 8],
 'fnlwgt': [8, 9],
 'education': [9, 25],
 'education-num': [25, 26],
 'marital-status': [26, 33],
 'occupation': [33, 47],
 'relationship': [47, 53],
 'race': [53, 58],
 'sex': [58, 60],
 'capital-gain': [60, 61],
 'capital-loss': [61, 62],
 'hours-per-week': [62, 63],
 'native-country': [63, 104],
 'income': [104, 106]}

In [31]:
#np_data = np_data[:,:104]
target_col = np.expand_dims(np.argmax(np_data[:,104:106], axis=-1), axis=-1)
np_data = np_data[:,:104]
np_data = np.hstack((np_data,target_col))
print(np_data.shape)

(30162, 105)


In [34]:
from sklearn.model_selection import train_test_split

y = np_data[:, 104]
X = np_data[:,:104]
train_size = 0.7
#X_tmp, X_test, y_tmp, y_test = train_test_split(X, y, train_size=train_size, random_state=0)
#X_train, X_vald, y_train, y_vald = train_test_split(X_tmp, y_tmp, train_size=0.9, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X_tmp, y_tmp, train_size=train_size, random_state=0)

print(f'Training set: {X_train.shape}')
print(f'Validation set: {X_vald.shape}')
print(f'Test set: {X_test.shape}')

Training set: (19001, 104)
Validation set: (2112, 104)
Test set: (9049, 104)


In [35]:
print(np.unique(y_train, return_counts=True))
print(np.unique(y_vald, return_counts=True))
print(np.unique(y_test, return_counts=True))

(array([0., 1.]), array([14287,  4714], dtype=int64))
(array([0., 1.]), array([1603,  509], dtype=int64))
(array([0., 1.]), array([6764, 2285], dtype=int64))


In [41]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=200)
gbc.fit(X_train, y_train)

GradientBoostingClassifier(n_estimators=200)

In [42]:
y_pred = gbc.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.862857774339706
[[6359  405]
 [ 836 1449]]
              precision    recall  f1-score   support

         0.0       0.88      0.94      0.91      6764
         1.0       0.78      0.63      0.70      2285

    accuracy                           0.86      9049
   macro avg       0.83      0.79      0.81      9049
weighted avg       0.86      0.86      0.86      9049

